# Exporting data from BigQuery to Google Cloud Storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

In [1]:
# Run the chown command to change the ownership of the repository
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Install the Google Cloud BigQuery library
%pip install google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 9.0 MB/s eta 0:00:01
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 1.3.1
    Uninstalling google-resumable-media-1.3.1:
      Successfully uninstalled google-resumable-media-1.3.1
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.20.0
    Uninstalling google-cloud-bigquery-2.20.0:
      Successfully uninstalled google-cloud-bigquery-2.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.40.0 requires google-resumable-media<2.0dev,>=1.3.0, but you have google-resumable-media 0.5.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Please ignore any incompatibility warnings and errors.
**Restart** the kernel to use updated packages. (On the Notebook menu, select Kernel > Restart Kernel > Restart).


In [3]:
# Importing necessary tensorflow library and printing the TF version.
import tensorflow as tf

print("Tensorflow version: ",tf.__version__)


Tensorflow version:  2.5.0


In [4]:
# The OS module in python provides functions for interacting with the operating system.
import os

from google.cloud import bigquery

Change the following cell as necessary:

In [10]:
# Change with your own bucket and project below:
BUCKET =  "qwiklabs-gcp-02-8c80cc2930da"
PROJECT = "qwiklabs-gcp-02-8c80cc2930da"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

## Create BigQuery tables

If you haven not already created a BigQuery dataset for our data, run the following cell:

In [11]:
# Initialize a BigQuery client.
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

# Create a new dataset with the `create_dataset()` method.
try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset created


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [12]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1552.49query/s]                        


""


Make the validation dataset be 1/10 the size of the training dataset.

In [13]:
%%bigquery

CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1443.16query/s]                        


""


## Export the tables as CSV files

In [14]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

# With `-l` option, gsutil will output additional information about each matching provider, bucket, subdirectory, or object.
gsutil ls -l $OUTDIR

Deleting current contents of gs://qwiklabs-gcp-02-8c80cc2930da/taxifare/data
Extracting training data to gs://qwiklabs-gcp-02-8c80cc2930da/taxifare/data
Extracting validation data to gs://qwiklabs-gcp-02-8c80cc2930da/taxifare/data
  88345235  2021-07-31T15:18:03Z  gs://qwiklabs-gcp-02-8c80cc2930da/taxifare/data/taxi-train-000000000000.csv
   8725746  2021-07-31T15:18:15Z  gs://qwiklabs-gcp-02-8c80cc2930da/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 2 objects, 97070981 bytes (92.57 MiB)


CommandException: 1 files/objects could not be removed.
Waiting on bqjob_rfdc3426a84c3f21_0000017afd24806c_1 ... (23s) Current status: DONE   
Waiting on bqjob_r57c844f6f1c3ed3c_0000017afd24e3d3_1 ... (3s) Current status: DONE   


In [15]:
# The cat command outputs the contents of one or more URLs to stdout.
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

18,2012-12-23 02:05:37 UTC,-73.978563,40.729148,-73.995454,40.683267,2,unused
2.5,2014-03-22 22:08:00 UTC,-73.987537,40.735972,-73.989617,40.736795,2,unused


Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.